# COGS 108 - Final Project


## Data Cleaning/Pre-processing

The first step in our project is to import our dependencies

In [62]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from collections import deque
from pytrends.request import TrendReq
from datetime import datetime
import time
from collections import defaultdict
from collections import OrderedDict

In [87]:
import sys
!{sys.executable} -m pip install pytrends

You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


First, we will define a helper function that we will use to return the celebrity name and death date, along with any other features we would like to extract. 

If we pass in any page that isn't a wikipedia page of a deceased notable celebrity (as per our definition), the function will return `None`.

In [2]:
def get_death_date(soup):
    table_selector = soup.select('table.infobox.vcard')

    if table_selector:
        table = table_selector[0]
        
        # Important features we would like to handle specially
        name = table.find('span', class_='fn')
        born = table.find('span', class_='bday')
        died = table.find('span', class_='dday deathdate')
        
        # Only return if the person has a name, has born data, and died
        if name and born and died:
            data =  {'Name': name.text, 'Born': born.text, 'Died': died.text}
            
            # Extract other features that we would like to keep.
            rows = table.find_all('tr')
            for row in rows:
                header = row.find('th')
                datum = row.find('td')
                if header and datum and header.text not in data:
                    data[header.text.replace('\xa0', ' ').strip()] = datum.get_text(' ', strip=True)
                    
            return data
        
    return None

Then, we will recursively traverse the links through Breadth First Search and visit all Wikipedia links reachable from the 'List of Celebrities' page. 

In [3]:
# Breadth First Search traversal.

wikipedia_url = 'https://en.wikipedia.org'
starting_endpoint = '/wiki/Lists_of_celebrities'
click_limit = 2

# Dictionary of visited links
# 'url' -> number of clicks taken to get to link
links_visited = {starting_endpoint: 0} 
# Queue of links to visit
links_queue = deque()
links_queue.appendleft(starting_endpoint)

# Create a buffer of dataframes
data_buffer = deque()

# Debug info
num_visited = 0

# Find all links on the page that lead to another wikipedia page.
while links_queue:
    link = links_queue.pop()
    page = requests.get(wikipedia_url + link)
    soup = BeautifulSoup(page.content, 'html.parser')
    num_visited += 1
    
    # Debug info
    if links_visited[link] >= click_limit - 1:
        num_links = len(links_visited)
        num_deads = len(data_buffer)
        percentage = format(100.0*(float(num_visited)/num_links), '.2f')
        print((percentage + "% - " + str(len(links_queue)) + " left, " + str(num_deads) + " recorded.").ljust(60),  end='\r', flush=True)
    else:
        print("Loading initial pages...", end='\r', flush=True)
    
    # See if page is of deceased person - if so, get the data
    results = get_death_date(soup)
    
    # Append to dataframe buffer if we've found a match
    if results:
        data_buffer.append(pd.DataFrame([results]))
    
    # Break early on click limit or if page isn't a list
    if links_visited[link] == click_limit or ("List" not in link and "Category:" not in link):
        continue
    
    content = soup.find('div', id='content')
    
    # Continue BFS on links in the page
    if content:
        for candidate in content.findAll('a'):
            if 'href' in candidate.attrs:
                link_in_page = urlparse(candidate.attrs['href']).path
                                
                # Add to queue if it's a link we haven't seen before.
                if link_in_page not in links_visited and link_in_page.startswith('/wiki/') and 'File' not in link_in_page:
                    links_visited[link_in_page] = links_visited[link] + 1
                    links_queue.appendleft(link_in_page)

# Concatenate all of the buffer into the dataframe that holds our celebrity data
df_celebrities = pd.concat(data_buffer)

Since this takes such a long time, we will run this once and cache it into a JSON file. All subsequent operations will be run off of the loaded JSON file.

In [112]:
with open('celebrities.json', 'w') as file:
    file.write(df_celebrities.to_json(orient='records'))

NameError: name 'df_celebrities' is not defined

Now, we can read the file back in as the JSON in order to circumvent the crawling process for future usage.

In [120]:
df_celebrities = pd.read_json("celebrities.json", orient="records")

In [121]:
date_objects = []

base_date = datetime.strptime("2004-01-01", "%Y-%m-%d")

for date in df_celebrities["Died"]:
    date_objects.append(datetime.strptime(date, "%Y-%m-%d"))
df_celebrities["Died"] = date_objects


df_valid = df_celebrities[df_celebrities["Died"] >= base_date]

Since pytrends records begin in 2004, we will discard any deaths that happened before that. This leaves us with 846 celebrities that died from 2004-01-01 to present.

In [122]:
#Search word to compare it to
comparator = "Grand Canyon"

#Open the pytrends request
pytrends = TrendReq(hl='en-US', tz=360)

#Create a list for the new column in the table
trends = []
comparator_average_trends = []
trend_ratio = []

#Find the relative trend at time of death compared to comparator
#Comparator relative trend value taken from 2004-01-01
def relative_trend(name, died):
    #Enter the name for searching the trend
    kw_list = [comparator, name]
    #Get a dataframe of the relative trends
    e_catcher = 0
    while e_catcher == 0:
        try:
            pytrends.build_payload(kw_list, timeframe='all')
            df_py = pytrends.interest_over_time()
            e_catcher = 1
        except:
            time.sleep(3600)
    cv = df_py[comparator].tolist()
    comparator_avg = sum(cv)/float(len(cv))
    trend = df_py[(df_py.index.month == died.month) & (df_py.index.year == died.year)][name].iloc[0]
    trend_ratio = trend/comparator_avg
    return trend, comparator_avg, trend_ratio
    

#Loop through and find all the trends relative to grand canyon
counter = 0
for name, died in zip(df_valid["Name"], df_valid["Died"]):
    #Check if there are more than two parts to the name
    name_split = name.split(" ")
    name_split_len = len(name_split)
    shortened_name = name_split[0] + " " + name_split[name_split_len-1]
    if  name_split_len > 2:
        long_rt, cvl, trl = relative_trend(name,died)
        short_rt, cvs, trs = relative_trend(shortened_name,died)
        if long_rt > short_rt:
            trends.append(long_rt)
            comparator_average_trends.append(cvl)
            trend_ratio.append(trl)
        else:
            trends.append(short_rt)
            comparator_average_trends.append(cvs)
            trend_ratio.append(trs)
    else:
        trend, cv, tr = relative_trend(name,died)
        trends.append(trend)
        comparator_average_trends.append(cv)
        trend_ratio.append(tr)
    counter += 1
    print(("Celebrity trends parsed = " + str(counter)), end='\r', flush=True)
df_valid['Trends'] = trends
df_valid['Comp_Average'] = comparator_average_trends
df_valid['Trend_Ratio'] = trend_ratio

/Users/joseph/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/joseph/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/joseph/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [123]:
with open('valid.json', 'w') as file:
    file.write(df_valid.to_json(orient='records'))

In [124]:
df_valid = pd.read_json("valid.json", orient="records")

In [125]:
df_valid["Died"] = pd.to_datetime(df_valid['Died'], unit="ms")
#Get rid of 2018, since its not a complete year
df_valid = df_valid[df_valid["Died"] < datetime.strptime("2018-01-01", "%Y-%m-%d")]

In [126]:
#Count the number of deaths per year
deaths_per_year = defaultdict(int)
for row in df_valid["Died"]:
    deaths_per_year[row.year] += 1
deaths_sorted = OrderedDict(sorted(deaths_per_year.items()))

In [127]:
#Print number of deaths per year sorted by deaths
for key,value in deaths_sorted.items():
    print( "Deaths in " + str(key) + " = " + str(value))

Deaths in 2004 = 56
Deaths in 2005 = 48
Deaths in 2006 = 56
Deaths in 2007 = 48
Deaths in 2008 = 58
Deaths in 2009 = 45
Deaths in 2010 = 62
Deaths in 2011 = 61
Deaths in 2012 = 58
Deaths in 2013 = 61
Deaths in 2014 = 60
Deaths in 2015 = 63
Deaths in 2016 = 78
Deaths in 2017 = 77


In [103]:
#Find the trends per year
trends_per_year = defaultdict(list)
for key,value in deaths_per_year.items():
    df = df_valid[(df_valid.Died >= datetime.strptime(str(key), "%Y")) & (df_valid.Died < datetime.strptime(str(key + 1), "%Y"))]
    trends_per_year[key] = df.Trends.tolist()

In [104]:
#Find the sum of trends and average of trends per year
trends_per_year_sum = defaultdict(int)
trends_per_year_avg = defaultdict(float)

for key,value in trends_per_year.items():
    tr = trends_per_year[key]
    trends_per_year_sum[key] = sum(tr)
    trends_per_year_avg[key] = sum(tr)/ float(len(tr))

In [105]:
trends_per_year_sum_ord = OrderedDict(sorted(trends_per_year_sum.items()))
trends_per_year_avg_ord = OrderedDict(sorted(trends_per_year_avg.items()))

In [106]:
for key,value in trends_per_year_sum_ord.items():
    print( "Trend sum in " + str(key) + " = " + str(value))

Trend sum in 2004 = -2862
Trend sum in 2005 = -2604
Trend sum in 2006 = -3329
Trend sum in 2007 = -2884
Trend sum in 2008 = -3060
Trend sum in 2009 = -2069
Trend sum in 2010 = -2992
Trend sum in 2011 = -3044
Trend sum in 2012 = -2297
Trend sum in 2013 = -3297
Trend sum in 2014 = -2630
Trend sum in 2015 = -3084
Trend sum in 2016 = -3120
Trend sum in 2017 = -3632


In [107]:
for key,value in trends_per_year_avg_ord.items():
    print( "Trend avg in " + str(key) + " = " + str(value))

Trend avg in 2004 = -51.107142857142854
Trend avg in 2005 = -54.25
Trend avg in 2006 = -59.44642857142857
Trend avg in 2007 = -60.083333333333336
Trend avg in 2008 = -52.758620689655174
Trend avg in 2009 = -45.977777777777774
Trend avg in 2010 = -48.25806451612903
Trend avg in 2011 = -49.90163934426229
Trend avg in 2012 = -39.60344827586207
Trend avg in 2013 = -54.049180327868854
Trend avg in 2014 = -43.833333333333336
Trend avg in 2015 = -48.95238095238095
Trend avg in 2016 = -40.0
Trend avg in 2017 = -47.16883116883117


In [108]:
for key,value in trends_per_year_avg_ord.items():
    print( "Deaths, Trend sum, Trend average in " + str(key) + " = " + str(deaths_per_year[key]) + ", " + str(trends_per_year_sum[key]) + ", " + str(trends_per_year_avg[key]))

Deaths, Trend sum, Trend average in 2004 = 56, -2862, -51.107142857142854
Deaths, Trend sum, Trend average in 2005 = 48, -2604, -54.25
Deaths, Trend sum, Trend average in 2006 = 56, -3329, -59.44642857142857
Deaths, Trend sum, Trend average in 2007 = 48, -2884, -60.083333333333336
Deaths, Trend sum, Trend average in 2008 = 58, -3060, -52.758620689655174
Deaths, Trend sum, Trend average in 2009 = 45, -2069, -45.977777777777774
Deaths, Trend sum, Trend average in 2010 = 62, -2992, -48.25806451612903
Deaths, Trend sum, Trend average in 2011 = 61, -3044, -49.90163934426229
Deaths, Trend sum, Trend average in 2012 = 58, -2297, -39.60344827586207
Deaths, Trend sum, Trend average in 2013 = 61, -3297, -54.049180327868854
Deaths, Trend sum, Trend average in 2014 = 60, -2630, -43.833333333333336
Deaths, Trend sum, Trend average in 2015 = 63, -3084, -48.95238095238095
Deaths, Trend sum, Trend average in 2016 = 78, -3120, -40.0
Deaths, Trend sum, Trend average in 2017 = 77, -3632, -47.168831168831

In [109]:
#Count the number of positive trend deaths per year
positive_deaths_per_year = defaultdict(int)
for death,trend in zip(df_valid["Died"],df_valid["Trends"]):
    if trend >= 0:
        positive_deaths_per_year[death.year] += 1
positive_deaths_sorted = OrderedDict(sorted(positive_deaths_per_year.items()))

In [111]:
for key,value in positive_deaths_sorted.items():
    print( "Positive Trend Deaths in " + str(key) + " = " + str(value))

Positive Trend Deaths in 2004 = 5
Positive Trend Deaths in 2005 = 3
Positive Trend Deaths in 2006 = 1
Positive Trend Deaths in 2008 = 4
Positive Trend Deaths in 2009 = 4
Positive Trend Deaths in 2010 = 7
Positive Trend Deaths in 2011 = 5
Positive Trend Deaths in 2012 = 8
Positive Trend Deaths in 2013 = 2
Positive Trend Deaths in 2014 = 8
Positive Trend Deaths in 2015 = 8
Positive Trend Deaths in 2016 = 11
Positive Trend Deaths in 2017 = 9
